Data Extraction

In [2]:
!pip install pyyaml

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install pandas


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ------- -------------------------------- 2.1/11.0 MB 13.7 MB/s eta 0:00:01
   ---------------------- ----------------- 6.3/11.0 MB 16.9 MB/s eta 0:00:01
   ---------------------------------------  10.7/11.0 MB 19.1 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 17.5 MB/s eta 0:00:00


In [8]:
import os
import yaml
import pandas as pd


month_folders = [
    r"C:\Users\judej\OneDrive\Desktop\Stock analaysis\Data\2023-10",
    r"C:\Users\judej\OneDrive\Desktop\Stock analaysis\Data\2023-11",
    r"C:\Users\judej\OneDrive\Desktop\Stock analaysis\Data\2023-12",
    r"C:\Users\judej\OneDrive\Desktop\Stock analaysis\Data\2024-01",
    r"C:\Users\judej\OneDrive\Desktop\Stock analaysis\Data\2024-02",
    r"C:\Users\judej\OneDrive\Desktop\Stock analaysis\Data\2024-03",
    r"C:\Users\judej\OneDrive\Desktop\Stock analaysis\Data\2024-04",
    r"C:\Users\judej\OneDrive\Desktop\Stock analaysis\Data\2024-05",
    r"C:\Users\judej\OneDrive\Desktop\Stock analaysis\Data\2024-06",
    r"C:\Users\judej\OneDrive\Desktop\Stock analaysis\Data\2024-07",
    r"C:\Users\judej\OneDrive\Desktop\Stock analaysis\Data\2024-08",
    r"C:\Users\judej\OneDrive\Desktop\Stock analaysis\Data\2024-09",
    r"C:\Users\judej\OneDrive\Desktop\Stock analaysis\Data\2024-10",
    r"C:\Users\judej\OneDrive\Desktop\Stock analaysis\Data\2024-11"
]


sector_csv = r"C:\Users\judej\OneDrive\Desktop\Stock analaysis\Data\Sector_data - Sheet1.csv"


individual_csv_dir = r"C:\Users\judej\OneDrive\Desktop\Stock analaysis\Main"
final_csv_path = os.path.join(individual_csv_dir, "all_data_with_sector.csv")
os.makedirs(individual_csv_dir, exist_ok=True)


sector_df = pd.read_csv(sector_csv)
sector_df.columns = sector_df.columns.str.strip()

sector_df['Ticker'] = sector_df['Symbol'].str.split(":").str[-1].str.strip()
sector_df['Company'] = sector_df['COMPANY'].str.strip()
sector_df['Sector'] = sector_df['sector'].str.strip()

sector_map = dict(zip(sector_df['Ticker'], sector_df['Sector']))
company_map = dict(zip(sector_df['Ticker'], sector_df['Company']))


ticker_corrections = {
    'ADANIENT': 'ADANIGREEN',
    'BHARTIARTL': 'AIRTEL',
    'TATACONSUM': 'TATACONSUMER',
    'BRITANNIA': 'NESTLEIND'
}


all_data = []

for month_path in month_folders:
    for file in os.listdir(month_path):
        if file.endswith(".yaml"):
            file_path = os.path.join(month_path, file)
            with open(file_path, "r") as f:
                day_data = yaml.safe_load(f)
                for entry in day_data:
                    raw_ticker = entry['Ticker']
                    corrected_ticker = ticker_corrections.get(raw_ticker, raw_ticker)
                    entry['Ticker'] = corrected_ticker
                    entry['Sector'] = sector_map.get(corrected_ticker, "Unknown")
                    entry['Company'] = company_map.get(corrected_ticker, "Unknown")
                    all_data.append(entry)


df = pd.DataFrame(all_data)

for ticker, group_df in df.groupby("Ticker"):
    output_path = os.path.join(individual_csv_dir, f"{ticker}.csv")
    group_df.to_csv(output_path, index=False)

df.to_csv(final_csv_path, index=False)



Data Frame creation For Requirements


In [9]:
import pandas as pd
import numpy as np
import os
file_path = r"C:\Users\judej\OneDrive\Desktop\Stock analaysis\Main\all_data_with_sector.csv"
output_dir = r"C:\Users\judej\OneDrive\Desktop\Stock analaysis\Main\Required Df"
os.makedirs(output_dir, exist_ok=True)

df = pd.read_csv(file_path)
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.to_period('M').astype(str)
df.sort_values(by=['Ticker', 'date'], inplace=True)
df['Prev_Close'] = df.groupby('Ticker')['close'].shift(1)
df['Daily_Return'] = (df['close'] - df['Prev_Close']) / df['Prev_Close']
df['Daily_Return'] = df['Daily_Return'].fillna(0)


volatility_df = df.groupby(['Ticker', 'Company'])['Daily_Return'].std().reset_index()
volatility_df.columns = ['Ticker', 'Company', 'Volatility']
volatility_df.to_csv(os.path.join(output_dir, "Volatility_Analysis.csv"), index=False)


df['Cumulative_Return'] = df.groupby('Ticker')['Daily_Return'].cumsum()
cumulative_df = df[['date', 'Ticker', 'Company', 'Cumulative_Return']]
cumulative_df.to_csv(os.path.join(output_dir, "Cumulative_Return.csv"), index=False)


yearly_return_df = df.groupby('Ticker').agg(
    Start_Close=('close', 'first'),
    End_Close=('close', 'last'),
    Sector=('Sector', 'last'),
    Company=('Company', 'last')
).reset_index()
yearly_return_df['Yearly_Return'] = (
    (yearly_return_df['End_Close'] - yearly_return_df['Start_Close']) / yearly_return_df['Start_Close']
)
sector_perf_df = yearly_return_df.groupby('Sector')['Yearly_Return'].mean().reset_index()
sector_perf_df.to_csv(os.path.join(output_dir, "Sector_wise_Performance.csv"), index=False)

pivot_company_df = df.groupby(['date', 'Company'])['Daily_Return'].mean().unstack()
correlation_company_df = pivot_company_df.corr()
correlation_company_df.to_csv(os.path.join(output_dir, "Correlation.csv"))


monthly_return_df = df.groupby(['month', 'Ticker', 'Company'])['Daily_Return'].sum().reset_index()

def top_bottom_ranked(group):
    top = group.nlargest(5, 'Daily_Return').copy()
    top['Rank'] = range(1, 6)
    bottom = group.nsmallest(5, 'Daily_Return').copy()
    bottom['Rank'] = range(-5, 0)
    return pd.concat([top, bottom])

monthly_top_bottom_ranked_df = (
    monthly_return_df.groupby('month', group_keys=False)
    .apply(top_bottom_ranked)
    .reset_index(drop=True)
)

monthly_top_bottom_ranked_df['month'] = pd.to_datetime(monthly_top_bottom_ranked_df['month']).dt.strftime('%B %Y')
monthly_top_bottom_ranked_df.to_csv(os.path.join(output_dir, "Month_wise_data.csv"), index=False)


print("Volatility_Analysis null \n", volatility_df.isnull().sum(), "\n")
print("Cumulative_Return null \n", cumulative_df.isnull().sum(), "\n")
print("Sector_wise_Performance null \n", sector_perf_df.isnull().sum(), "\n")
print("Correlation null \n", correlation_company_df.isnull().sum(), "\n")
print("Month_wise_data null \n", monthly_top_bottom_ranked_df.isnull().sum(), "\n")

Volatility_Analysis null 
 Ticker        0
Company       0
Volatility    0
dtype: int64 

Cumulative_Return null 
 date                 0
Ticker               0
Company              0
Cumulative_Return    0
dtype: int64 

Sector_wise_Performance null 
 Sector           0
Yearly_Return    0
dtype: int64 

Correlation null 
 Company
ADANI ENTERPRISES      0
ADANI PORTS & SEZ      0
APOLLO HOSPITALS       0
ASIAN PAINTS           0
AXIS BANK              0
BAJAJ AUTO             0
BAJAJ FINANCE          0
BAJAJ FINSERV          0
BHARAT ELECTRONICS     0
BHARTI AIRTEL          0
BPCL                   0
CIPLA                  0
COAL INDIA             0
DR. REDDYS LAB         0
EICHER MOTORS          0
GRASIM                 0
HCL TECHNOLOGIES       0
HDFC BANK              0
HDFC LIFE INSURANCE    0
HERO MOTOCORP          0
HINDALCO               0
HINDUSTAN UNILEVER     0
ICICI BANK             0
INDUSIND BANK          0
INFOSYS                0
ITC                    0
JSW STEEL        

C:\Users\judej\AppData\Local\Temp\ipykernel_12140\3495646223.py:55: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(top_bottom_ranked)


Data Sql Insertion


In [17]:
pip install mysql-connector-python


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
   ------------ --------------------------- 5.2/16.4 MB 33.2 MB/s eta 0:00:01
   ---------------------------- ----------- 11.8/16.4 MB 31.6 MB/s eta 0:00:01
   ---------------------------------------- 16.4/16.4 MB 31.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [10]:
import mysql.connector as mysql
import pandas as pd
import os


connection = mysql.connect(
    host='localhost',
    user='root',
    password='root',
    database='stock_analysis_db',
    port=3306
)
cursor = connection.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS all_data_with_sector (
    Ticker VARCHAR(20),
    close FLOAT,
    date DATETIME,
    high FLOAT,
    low FLOAT,
    month VARCHAR(20),
    open FLOAT,
    volume BIGINT,
    Sector VARCHAR(100),
    Company VARCHAR(100)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Volatility_Analysis (
    Ticker VARCHAR(20),
    Company VARCHAR(100),
    Volatility FLOAT
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Cumulative_Return (
    date DATETIME,
    Ticker VARCHAR(20),
    Company VARCHAR(100),
    Cumulative_Return FLOAT
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Sector_wise_Performance (
    Sector VARCHAR(100),
    Yearly_Return FLOAT
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Month_wise_data (
    month VARCHAR(20),
    Ticker VARCHAR(20),
    Company VARCHAR(100),
    Daily_Return FLOAT,
    `Rank` INT
);
""")


main_csv = r"C:\Users\judej\OneDrive\Desktop\Stock analaysis\Main\all_data_with_sector.csv"
output_dir = r"C:\Users\judej\OneDrive\Desktop\Stock analaysis\Main\Required Df"

df_sector = pd.read_csv(main_csv)
df_volatility = pd.read_csv(os.path.join(output_dir, "Volatility_Analysis.csv"))
df_cum_return = pd.read_csv(os.path.join(output_dir, "Cumulative_Return.csv"))
df_sector_perf = pd.read_csv(os.path.join(output_dir, "Sector_wise_Performance.csv"))
df_monthly = pd.read_csv(os.path.join(output_dir, "Month_wise_data.csv"))


df_sector.columns = df_sector.columns.str.strip()
df_volatility.columns = df_volatility.columns.str.strip()
df_cum_return.columns = df_cum_return.columns.str.strip()
df_sector_perf.columns = df_sector_perf.columns.str.strip()
df_monthly.columns = df_monthly.columns.str.strip()



insert_sector = """
INSERT INTO all_data_with_sector 
(Ticker, close, date, high, low, month, open, volume, Sector, Company)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""
for _, row in df_sector.iterrows():
    cursor.execute(insert_sector, tuple(row))

insert_vol = """
INSERT INTO Volatility_Analysis (Ticker, Company, Volatility)
VALUES (%s, %s, %s);
"""
for _, row in df_volatility.iterrows():
    cursor.execute(insert_vol, (row['Ticker'], row['Company'], row['Volatility']))

insert_cum = """
INSERT INTO Cumulative_Return (date, Ticker, Company, Cumulative_Return)
VALUES (%s, %s, %s, %s);
"""
for _, row in df_cum_return.iterrows():
    cursor.execute(insert_cum, (row['date'], row['Ticker'], row['Company'], row['Cumulative_Return']))

insert_sector_perf = """
INSERT INTO Sector_wise_Performance (Sector, Yearly_Return)
VALUES (%s, %s);
"""
for _, row in df_sector_perf.iterrows():
    cursor.execute(insert_sector_perf, (row['Sector'], row['Yearly_Return']))

insert_monthly = """
INSERT INTO Month_wise_data (month, Ticker, Company, Daily_Return, `Rank`)
VALUES (%s, %s, %s, %s, %s);
"""
for _, row in df_monthly.iterrows():
    cursor.execute(insert_monthly, (
        row['month'], row['Ticker'], row['Company'], row['Daily_Return'], row['Rank']
    ))

# === Finalize ===
connection.commit()
cursor.close()
connection.close()


